In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

In [ ]:
## imports and object instantiation
import json, time, nest_asyncio, json, itertools, sys, threading
from pystackql import StackQL
import pandas as pd
from IPython.display import clear_output, display, Markdown, HTML
from ipytree import Tree, Node

stackql = StackQL()
nest_asyncio.apply()

In [ ]:
## functions

def display_cards(cards_data):
    cards_html = ''
    
    for title, value in cards_data:
        card_template = f"""
        <div style="
            border: 1px solid #e3e3e3;
            border-radius: 4px;
            padding: 20px;
            display: inline-block;
            margin: 5px;
            text-align: center;
            width: 150px;
            background-color: #f7f7f7;">
            <h4 style="margin: 5px 0;">{title}</h4>
            <span style="font-size: 30px; font-weight: bold; color: red;">{value}</span>
        </div>
        """
        cards_html += card_template
    
    display(HTML(cards_html))

def get_icon(resType):
    if resType == "project":
        return 'codepen'
    else:
        return resType

def print_overwrite(message):
    clear_output(wait=True)
    print(message)

def spinning_cursor():
    spinner = itertools.cycle(['-', '/', '|', '\\'])
    while getattr(threading.current_thread(), "do_run", True):
        sys.stdout.write(next(spinner))
        sys.stdout.flush()
        time.sleep(0.1)
        sys.stdout.write('\b')

def build_tree_node(df, parent_name, parent_node=None):
    children = df[df['parentDisplayName'] == parent_name]
    
    for _, child in children.iterrows():
        child_node = Node(child['displayName'], opened=False, icon=get_icon(child['resType']))
        if parent_node:
            parent_node.add_node(child_node)
        build_tree_node(df, child['displayName'], child_node)

def explode_json_list_col(input_df, col_to_explode, exploded_col):
    output_df = input_df
    output_df[col_to_explode] = output_df[col_to_explode].apply(json.loads)
    output_df = output_df.explode(col_to_explode).rename(columns={col_to_explode: exploded_col})
    return output_df

In [ ]:
# get all folders and projects function
def get_resources_recursive(entity_id, get_projects_query_fn, get_folders_query_fn, parent_display_name='organization'):
    resources = []

    # Query for projects
    print_overwrite(f"Searching {entity_id} for projects...")
    project_query = get_projects_query_fn(entity_id)
    project_results = json.loads(stackql.execute(project_query))

    if isinstance(project_results, list):
        print_overwrite(f"Found {len(project_results)} projects in {entity_id}")
        for proj in project_results:
            proj["parentDisplayName"] = parent_display_name
            proj["resType"] = "project"
            resources.append(proj)

    # Query for folders
    print_overwrite(f"Searching {entity_id} for folders...")
    folder_query = get_folders_query_fn(entity_id)
    folder_results = json.loads(stackql.execute(folder_query))

    if isinstance(folder_results, list):
        print_overwrite(f"Found {len(folder_results)} folders in {entity_id}")
        for folder in folder_results:
            folder["parentDisplayName"] = parent_display_name
            folder["resType"] = "folder"
            resources.append(folder)

            # Fetch resources under this folder
            if 'name' in folder:
                resources.extend(get_resources_recursive(folder['name'], get_projects_query_fn, get_folders_query_fn, folder['displayName']))

    return resources

def get_all_resources(get_projects_query, get_folders_query):
    start_time = time.time()
    
    # Start with the root organization to get all resources
    all_resources = get_resources_recursive("organizations/%s" % (org_id), get_projects_query, get_folders_query)
    
    # Convert list to dataframe and filter
    resources_df = (pd.DataFrame(all_resources)
                    .loc[lambda df: df['error'].isna()]
                    .drop('error', axis=1, errors='ignore'))
    
    # Create root node and build the tree
    root = Node("organization", opened=False, icon='building')
    build_tree_node(resources_df, "organization", root)
    
    # Display the tree
    tree = Tree(nodes=[root])
    
    # Calculate metrics and display
    elapsed_time = round(time.time() - start_time)
    num_folders = resources_df.query("resType == 'folder'").shape[0]
    num_projects = resources_df.query("resType == 'project'").shape[0]
    projects_df = resources_df.query("resType == 'project'")['projectId'].dropna().to_frame()
    projects = projects_df['projectId'].tolist()
    
    print(f"Total elapsed time: {elapsed_time} seconds")
    cards_data = [("Number of Projects", num_projects), ("Number of Folders", num_folders)]
    display_cards(cards_data)
    
    return resources_df, projects_df, projects, tree


In [ ]:
# project iam bindings function
def get_project_bindings(queries):
    # Start the spinning cursor on a separate thread
    spinner_thread = threading.Thread(target=spinning_cursor)
    spinner_thread.start()

    start_time = time.time()

    res = stackql.executeQueriesAsync(queries)

    bindings_df = (
        explode_json_list_col(pd.read_json(json.dumps(res)), 'members', 'member')
        .assign(**{
            'member_type': lambda x: x['member'].str.split(':', n=1).str[0],
            'member_email': lambda x: x['member'].str.split(':', n=1).str[1]
        })
        .drop('member', axis=1)
    )

    number_of_rows = bindings_df.shape[0]
    elapsed_time = round(time.time() - start_time)

    # Stop the spinner
    spinner_thread.do_run = False
    spinner_thread.join()

    print(f"Found {number_of_rows} bindings in {elapsed_time} seconds")

    return bindings_df

In [ ]:
# regions and zones function
def get_all_regions_and_zones(projects_df, queries):

    # Start the spinning cursor on a separate thread
    spinner_thread = threading.Thread(target=spinning_cursor)
    spinner_thread.start()

    start_time = time.time()

    all_results = []
    for query in queries:
        res = stackql.execute(query)
        try:
            parsed_result = json.loads(res)
            all_results.extend(parsed_result)
        except json.JSONDecodeError:
            print(f"Failed to parse result from query: {query}")
            print(f"Raw result: {res}")
            
    zones_df = (
        pd.DataFrame(all_results)
        .loc[lambda x: x['error'].isnull()]
        .drop('error', axis=1)
        .drop_duplicates()
    )
    
    regions_df = pd.DataFrame(zones_df['region'].unique(), columns=['region'])
    # For every combination of project and region
    projects_regions_df = projects_df.assign(key=1).merge(regions_df.assign(key=1), on='key').drop('key', axis=1)
    # For every combination of project and zone
    projects_zones_df = projects_df.assign(key=1).merge(zones_df.assign(key=1), on='key').drop(['key', 'region'], axis=1)
    
    # python list variables
    regions = regions_df['region'].tolist()
    zones = zones_df['name'].tolist()
    projects_regions = projects_regions_df.to_dict(orient='records')
    projects_zones = projects_zones_df.to_dict(orient='records')

    number_of_rows = zones_df.shape[0]
    number_of_projects = projects_df.shape[0]
    elapsed_time = round(time.time() - start_time)

    # Stop the spinner
    spinner_thread.do_run = False
    spinner_thread.join()

    print(f"Found {number_of_rows} zones across {number_of_projects} projects in {elapsed_time} seconds")

    return regions_df, zones_df, regions, zones, projects_regions, projects_zones